In [3]:
import numpy as np

def make_f(A, B, C, D):
    # compute returns
    R = np.column_stack([
        np.diff(A) / A[:-1],
        np.diff(B) / B[:-1],
        np.diff(C) / C[:-1],
        np.diff(D) / D[:-1]
    ])
    n = len(R)
    
    def f(wA, wB, wC):
        wD = 1 - wA - wB - wC
        w = np.array([wA, wB, wC, wD])
        r_p = R @ w
        mean_rp = np.mean(r_p)
        return np.sum((r_p - mean_rp)**2)  # (n-1)*σ² = f
    return f

A = [100,102,101,103,105,104,106,108,109,111,112,113]
B = [ 50, 49, 50, 52, 51, 53, 55, 56, 57, 58, 60, 61]
C = [ 80, 81, 82, 82, 84, 85, 86, 87, 88, 89, 91, 92]
D = [120,121,119,118,120,122,121,123,124,126,127,129]

f = make_f(A, B, C, D)
print(f)


<function make_f.<locals>.f at 0x0000023C1DA36340>


In [24]:
import numpy as np
import sympy as sp
from IPython.display import Math, display
import math

def sigma_from_f(f, n):
    return math.sqrt(f / (n - 1))

def make_f_expr(A, B, C, D):
    # compute returns
    R = np.column_stack([
        np.diff(A) / A[:-1],
        np.diff(B) / B[:-1],
        np.diff(C) / C[:-1],
        np.diff(D) / D[:-1]
    ])
    n = len(R)

    # define symbols
    wA, wB, wC = sp.symbols('w_A w_B w_C')
    wD = 1 - wA - wB - wC
    w = [wA, wB, wC, wD]

    # compute portfolio returns one by one
    r_p = []
    for i in range(n):
        term = sum(R[i, j] * w[j] for j in range(4))
        r_p.append(term)

    # compute mean manually
    mean_rp = sum(r_p) / n

    # compute f = sum((r_p[i] - mean_rp)**2)
    f_expr = sum((r_p[i] - mean_rp)**2 for i in range(n))

    return sp.simplify(f_expr)

# Example usage
A = [100,102,101,103,105,104,106,108,109,111,112,113]
B = [50,49,50,52,51,53,55,56,57,58,60,61]
C = [80,81,82,82,84,85,86,87,88,89,91,92]
D = [120,121,119,118,120,122,121,123,124,126,127,129]
n = len(A) - 1

f_expr = make_f_expr(A, B, C, D)

# assuming f_expr already computed
f_latex = sp.latex(sp.N(f_expr, 3))

display(Math(f"{f_latex}"))

<IPython.core.display.Math object>

In [ ]:
val = f_expr.subs({'w_A': 0.2, 'w_B': 0.3, 'w_C': 0.1}).evalf()
print(val)

0.000399833086041604


In [19]:
wA, wB, wC = sp.symbols('w_A w_B w_C')
solutions = sp.solve([
    sp.diff(f_expr, wA),
    sp.diff(f_expr, wB),
    sp.diff(f_expr, wC)
], (wA, wB, wC))
print(solutions)


{w_A: 0.221534753830334, w_B: 0.158254957754298, w_C: 0.576750752650851}


In [21]:
val = f_expr.subs({'w_A': 0.221534753830334, 'w_B': 0.158254957754298, 'w_C': 0.576750752650851}).evalf()
val

0.000146215942403613

In [25]:
sigma_from_f(val, n)

0.0038238193263230016

In [36]:
coeff = f_expr.coeff(wA**2)
f_expr = f_expr - coeff*wA**2 - 0.003*wA**2
f_expr


-0.003*w_A**2 + 0.00262281268914675*w_A*w_B + 0.00178321546154063*w_A*w_C - 0.0024409014879152*w_A + 0.007715974228636*w_B**2 + 0.00299822983139518*w_B*w_C - 0.00475245782648269*w_B + 0.00127526367993523*w_C**2 - 0.00234054750806584*w_C + 0.00146759447225388

In [41]:
wA, wB, wC = sp.symbols('w_A w_B w_C')
solutions = sp.solve([
    sp.diff(f_expr, wA),
    sp.diff(f_expr, wB),
    sp.diff(f_expr, wC)
], (wA, wB, wC))
print(solutions)


{w_A: -0.0984866669542998, w_B: 0.172407299680589, w_C: 0.783858968830379}


In [43]:
f_expr.subs({'w_A': 100, 'w_B': 0.0, 'w_C': 0.0}).evalf()

-30.2426225543193